In [2]:
import requests, torch
from config import config
from van import Fan

In [4]:
fan = Fan(config['ckey'], config['csecret'])
fan.xauth(config['myuser'], config['mypass'])
tl = fan.public_timeline

In [103]:
fan.request('GET', 'account/rate_limit_status')

{'hourly_limit': 1500,
 'remaining_hits': 1428,
 'reset_time': 'Thu Dec 06 20:45:27 +0000 2018',
 'reset_time_in_seconds': 1544129127}

In [58]:
wordlist = ['清华大学', '微信支付', '中国银行']

In [120]:
load_all()

In [118]:
wordlist

['清华大学', '微信支付', '中国银行', '贸易战']

In [123]:
import pickle

In [127]:
def save_all():
    with open('./wordlist.pkl', 'wb') as f:
        pickle.dump(wordlist,f,True)
    with open('./state.pkl', 'wb') as f:
        pickle.dump(state,f,True)
    with open('./mem.pkl', 'wb') as f:
        pickle.dump(mem,f,True)

In [128]:
save_all()

In [59]:
state = {}

In [98]:
mem = {}

In [65]:
def save_all():
    torch.save(wordlist,'./wordlist.pkl')
    torch.save(state,'./state.pkl')
    torch.save(mem,'./mem.pkl')

In [66]:
def load_all():
    global wordlist
    wordlist = torch.load('./wordlist.pkl')
    global state
    state = torch.load('./state.pkl')
    global mem
    mem = torch.load('./mem.pkl')
    

In [100]:
# save_all()

In [110]:
def QiRiNianHua(seq):
    ret = [0]*370
    for i in range(len(seq)):
        if seq[i]>0:
            for j in range(7):
                ret[i+j] += seq[i]
    for i in range(len(ret)):
        ret[i] *= 52
    return ret[:367]
    

In [111]:
def calc_state(word, result):
    cnt = [0]*367
    for x in result:
        if datetimefromstr(x).year == 2018:
            idx = get_day_num(x)
            cnt[idx] += 1
    cnt = QiRiNianHua(cnt)
    return cnt[1:get_today_num()+1]

In [112]:
def update(word):
    if word in mem and 'lastupdate' in mem[word] and mem[word]['lastupdate'] == get_today_num():
        state[word] = calc_state(word, mem[word]['status'])
        return
    if word not in mem:
        mem[word] = {}
    if 'status' not in mem[word]:
        mem[word]['status'] = []
    if 'lastid' not in mem[word]:
        mem[word]['lastid'] = -1
    lastid = mem[word]['lastid']
    latest = fan.request('GET', 'search/public_timeline', {'q':word, 'count':60})
    cnt = 200
    while cnt > 0 and len(latest)>0 and latest[-1]['rawid'] > lastid and datetimefromstr(latest[-1]['created_at']).year >= 2018:
        cnt -= 1
        print(200-cnt)
        latest += fan.request('GET', 'search/public_timeline', {'q':word, 'count':60, 'max_id':latest[-1]['id']})[1:]
    
    for x in latest:
        if x['rawid']> lastid:
            mem[word]['status'].append(x['created_at'])
    mem[word]['lastid'] = latest[0]['rawid']
    mem[word]['lastupdate'] = get_today_num()
    
    state[word] = calc_state(word, mem[word]['status'])
    
    

In [113]:
def update_all():
    load_all()
#     global mem
#     mem = {}
    for w in wordlist:
        print(w)
        update(w)
    save_all()

In [114]:
update_all()

清华大学
1
2
3
4
微信支付
1
2
3
4
5
6
7
中国银行
1


In [45]:
result = fan.request('GET', 'search/public_timeline', {'q':'微众银行', 'count':60})
result

[{'created_at': 'Tue Dec 04 06:45:13 +0000 2018',
  'favorited': False,
  'id': '0QkTtEa-YEU',
  'in_reply_to_screen_name': '',
  'in_reply_to_status_id': '',
  'in_reply_to_user_id': '',
  'is_self': False,
  'location': 'SZ',
  'rawid': 233774916,
  'source': '<a href="http://fan.zico.im" target="_blank">有饭</a>',
  'text': '微信零钱支付达到每年限额之后一般你们都怎么操作？开通&lt;strong&gt;微众银行&lt;/strong&gt;还是放弃微信支付呢？',
  'truncated': False,
  'user': {'birthday': '',
   'created_at': 'Sun Jan 31 18:02:19 +0000 2016',
   'description': 'Stuck in reverse.',
   'favourites_count': 427,
   'followers_count': 63,
   'following': False,
   'friends_count': 29,
   'gender': '',
   'id': '~qdL1t_GhTVU',
   'location': 'SZ',
   'name': 'DDDzX',
   'notifications': False,
   'photo_count': 184,
   'profile_background_color': '#232323',
   'profile_background_image_url': 'http://static.fanfou.com/img/bg/19.png',
   'profile_background_tile': True,
   'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e

In [14]:
import datetime

In [13]:
s = result[0]['created_at']

In [24]:
print(s)

Thu Dec 06 16:06:58 +0000 2018


In [26]:
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')

In [34]:
def get_day_num(s):
    return int(datetimefromstr(s).strftime('%j'))

In [36]:
def get_today_num():
    return int(datetime.datetime.today().strftime('%j'))

In [37]:
get_day_num(s)

340

In [38]:
get_today_num()

341

In [33]:
int(datetimefromstr(s).strftime('%j'))

340

In [64]:
datetime.datetime.now().year

2018

In [8]:
help(fan.request)

Help on method request in module van:

request(method, endpoint, params=None, data=None, files=None, **kwargs) method of van.Fan instance
    发出请求



In [39]:
def GenSeq(result):
    cnt = [0]*367
    for x in result:
        idx = get_day_num(x['created_at'])
        cnt[idx] += 1
    return cnt

In [46]:
seq = GenSeq(result)

In [50]:
for i in range(365):
    if seq[i]>0:
        print(i,seq[i])

7 1
9 2
10 1
12 2
13 1
17 1
22 1
27 1
28 1
36 1
43 1
50 1
54 1
59 3
61 3
73 2
82 1
90 1
108 2
113 1
117 1
122 1
124 2
125 1
134 2
155 1
157 1
167 1
178 1
185 1
191 1
203 1
219 1
243 1
252 1
257 1
263 1
268 1
273 1
307 1
314 1
322 1
331 1
332 1
338 1
341 1
352 1
354 1
355 1
361 1


In [51]:
def QiRiNianHua(seq):
    ret = [0]*370
    for i in range(len(seq)):
        if seq[i]>0:
            for j in range(7):
                ret[i+j] += seq[i]
    for i in range(len(ret)):
        ret[i] *= 52
    return ret[:367]
    

In [52]:
seq7 = QiRiNianHua(seq)

In [55]:
for i in range(365):
    if seq7[i]>0:
        print(i,seq7[i])

7 52
8 52
9 156
10 208
11 208
12 312
13 364
14 312
15 312
16 208
17 208
18 208
19 104
20 52
21 52
22 104
23 104
24 52
25 52
26 52
27 104
28 156
29 104
30 104
31 104
32 104
33 104
34 52
36 52
37 52
38 52
39 52
40 52
41 52
42 52
43 52
44 52
45 52
46 52
47 52
48 52
49 52
50 52
51 52
52 52
53 52
54 104
55 104
56 104
57 52
58 52
59 208
60 208
61 312
62 312
63 312
64 312
65 312
66 156
67 156
73 104
74 104
75 104
76 104
77 104
78 104
79 104
82 52
83 52
84 52
85 52
86 52
87 52
88 52
90 52
91 52
92 52
93 52
94 52
95 52
96 52
108 104
109 104
110 104
111 104
112 104
113 156
114 156
115 52
116 52
117 104
118 104
119 104
120 52
121 52
122 104
123 104
124 156
125 208
126 208
127 208
128 208
129 156
130 156
131 52
134 104
135 104
136 104
137 104
138 104
139 104
140 104
155 52
156 52
157 104
158 104
159 104
160 104
161 104
162 52
163 52
167 52
168 52
169 52
170 52
171 52
172 52
173 52
178 52
179 52
180 52
181 52
182 52
183 52
184 52
185 52
186 52
187 52
188 52
189 52
190 52
191 104
192 52
193 52
194 5